# Project 03 - Taxi in OpenAI
## [Tutorial]
https://www.youtube.com/watch?v=q2ZOEFAaaI0

# 0. Import Dependencies

In [1]:
!pip install tensorflow==2.3.0 --user
!pip install gym

  Using cached tensorflow-2.3.0-cp37-cp37m-win_amd64.whl (342.5 MB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 1.15.0 requires gast==0.2.2, but you have gast 0.3.3 which is incompatible.
tensorflow-gpu 1.15.0 requires tensorboard<1.16.0,>=1.15.0, but you have tensorboard 2.8.0 which is incompatible.
tensorflow-gpu 1.15.0 requires tensorflow-estimator==1.15.1, but you have tensorflow-estimator 2.3.0 which is incompatible.


      Successfully uninstalled tensorboard-1.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [2]:
import numpy as np
import gym
import random

# 1. Test Random Environment

In [3]:
environment_name = 'Taxi-v3'

In [28]:
env = gym.make(environment_name)
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [25]:
episodes = 1
for episode in range (1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

env.close()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:

# 2. Create the Q-table and initialize it

In [31]:
action_size = env.action_space.n
print("Action size ", action_size)

state_size= env.observation_space.n
print("State size ", state_size)

Action size  6
State size  500


## [Remark]
Numpy to create matrix with size (row, column)

In [32]:
qtable = np.zeros ((state_size, action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


# 3. Create the hyperparamters

In [62]:
total_epsiodes = 500       # Total episodes
total_test_episodes = 10    # Total test episodes
max_steps = 99               # Max steps per episode

learning_rate = 0.7          # Learning rate
gamma = 0.618                # Discoutning rate

# Exploration parameters
# Start with exploration (random) then exploitation (follow max Q)

epsilon = 1.0                # Exploration rate
max_epsilon = 1.0            # Exploration probability at strt
min_epsilon = 0.01           # Minimum exploration probability
decay_rate = 0.01            # Exponential decay rate for exploration prob

# 4. The Q learning algorithm

In [57]:
# For life or unitl learning is stopped
for episode in range(total_epsiodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    
    for step in range(max_steps):
        # Choose an action a in the current world state (s)
        ## First we randomize a number for exploration or explotiation
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> explotiation (taking the biggest Q value for this state)
        if exp_exp_tradeoff  > epsilon:
            action = np.argmax(qtable[state,:])
        
        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome staet(s') and reward (r)
        new_state, reward, done, info = env.step(action)
        
        # Update Q(s,a) + lr [R(s,a) + gamma * max Q(s', a')  - Q(s,a)]
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * 
                                                                         np.max(qtable[new_state, :]) - qtable[state, action])
        
        # Our new state is state
        state = new_state
        
        # If done : finish episode
        if done == True:
            break
    
    episode += 1
    
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    
    print("Epsiode: ", episode, " Reward:", reward, "Epsilon ", epsilon)

Epsiode:  1  Reward: -1 Epsilon  0.9901493354116764
Epsiode:  2  Reward: -1 Epsilon  0.9803966865736877
Epsiode:  3  Reward: -1 Epsilon  0.970741078213023
Epsiode:  4  Reward: -10 Epsilon  0.9611815447608
Epsiode:  5  Reward: -10 Epsilon  0.9517171302557069
Epsiode:  6  Reward: -1 Epsilon  0.9423468882484062
Epsiode:  7  Reward: -1 Epsilon  0.9330698817068888
Epsiode:  8  Reward: -1 Epsilon  0.9238851829227694
Epsiode:  9  Reward: -1 Epsilon  0.9147918734185159
Epsiode:  10  Reward: -1 Epsilon  0.9057890438555999
Epsiode:  11  Reward: -1 Epsilon  0.896875793943563
Epsiode:  12  Reward: -1 Epsilon  0.888051232349986
Epsiode:  13  Reward: 20 Epsilon  0.8793144766113556
Epsiode:  14  Reward: -1 Epsilon  0.8706646530448178
Epsiode:  15  Reward: -10 Epsilon  0.8621008966608072
Epsiode:  16  Reward: 20 Epsilon  0.8536223510765493
Epsiode:  17  Reward: -1 Epsilon  0.8452281684304199
Epsiode:  18  Reward: 20 Epsilon  0.8369175092971592
Epsiode:  19  Reward: -1 Epsilon  0.8286895426039287
Epsio

Epsiode:  251  Reward: 20 Epsilon  0.09045555684848275
Epsiode:  252  Reward: 20 Epsilon  0.08965501068203711
Epsiode:  253  Reward: 20 Epsilon  0.08886243008303903
Epsiode:  254  Reward: 20 Epsilon  0.08807773579276819
Epsiode:  255  Reward: 20 Epsilon  0.08730084934114159
Epsiode:  256  Reward: 20 Epsilon  0.08653169303886674
Epsiode:  257  Reward: 20 Epsilon  0.0857701899696724
Epsiode:  258  Reward: 20 Epsilon  0.08501626398261702
Epsiode:  259  Reward: 20 Epsilon  0.0842698396844737
Epsiode:  260  Reward: 20 Epsilon  0.08353084243219053
Epsiode:  261  Reward: 20 Epsilon  0.08279919832542652
Epsiode:  262  Reward: 20 Epsilon  0.08207483419916123
Epsiode:  263  Reward: 20 Epsilon  0.08135767761637844
Epsiode:  264  Reward: 20 Epsilon  0.08064765686082219
Epsiode:  265  Reward: 20 Epsilon  0.0799447009298253
Epsiode:  266  Reward: 20 Epsilon  0.0792487395272088
Epsiode:  267  Reward: 20 Epsilon  0.07855970305625253
Epsiode:  268  Reward: 20 Epsilon  0.07787752261273513
Epsiode:  269 

Epsiode:  411  Reward: 20 Epsilon  0.026243696781029717
Epsiode:  412  Reward: 20 Epsilon  0.026082069297530375
Epsiode:  413  Reward: 20 Epsilon  0.02592205003436255
Epsiode:  414  Reward: 20 Epsilon  0.025763622989466578
Epsiode:  415  Reward: 20 Epsilon  0.025606772320005942
Epsiode:  416  Reward: 20 Epsilon  0.025451482340783003
Epsiode:  417  Reward: 20 Epsilon  0.02529773752267042
Epsiode:  418  Reward: 20 Epsilon  0.025145522491058264
Epsiode:  419  Reward: 20 Epsilon  0.02499482202431651
Epsiode:  420  Reward: 20 Epsilon  0.024845621052272927
Epsiode:  421  Reward: 20 Epsilon  0.024697904654705963
Epsiode:  422  Reward: 20 Epsilon  0.024551658059852766
Epsiode:  423  Reward: 20 Epsilon  0.024406866642931964
Epsiode:  424  Reward: 20 Epsilon  0.024263515924681227
Epsiode:  425  Reward: 20 Epsilon  0.024121591569909263
Epsiode:  426  Reward: 20 Epsilon  0.023981079386062323
Epsiode:  427  Reward: 20 Epsilon  0.023841965321804916
Epsiode:  428  Reward: 20 Epsilon  0.02370423546561

In [58]:
import pandas as pd
qtable_pd = pd.DataFrame(qtable)
qtable_pd

,0,1,2,3,4,5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-2.504215,-2.434005,-2.504215,-2.434005,-2.320397,-11.434005
2,-1.839102,-1.357770,-1.839102,-1.357770,-0.578916,-10.357770
3,-2.434005,-2.320397,-2.434005,-2.320397,-2.136565,-11.320397
4,-2.611473,-2.613837,-2.611473,-2.613853,-11.554464,-11.198640
...,...,...,...,...,...,...
495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
496,-2.320404,-2.136565,-2.320397,-2.136565,-11.320324,-11.320372
497,-0.694826,0.681366,-0.961488,-1.358242,-9.579593,-9.509285
498,-2.320397,-2.136565,-2.320397,-2.136565,-11.320394,-11.320397


# 5. Use our Q-table to play Taxi!

In [65]:
env.reset()
rewards = []

for epsiode in range(total_test_episodes):
    
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        env.render()
        # Take the action (index) that the maximum expected furture reward given that state
        action = np.argmax(qtable[state, :])
        
        new_state, reward, done, info = env.step(action)
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            print("Epsiode: ", epsiode)
            print ("Score", total_rewards)
            break
        state = new_state

env.close()
print("Score over time: " + str(sum(rewards)/total_test_episodes))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
|_: | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: